In [1]:
import geopandas as gpd

In [2]:
data_gpd=gpd.read_file('pop_dens_s2_5100.gpkg')

In [3]:
##Check cluster centre
data_cl_centre=data_gpd.query('pop_built_s2>=1500')

In [4]:
from pysal.lib import weights

In [5]:
wr = weights.contiguity.Rook.from_dataframe(data_cl_centre)
data_cl_centre['cluster_label']=wr.component_labels

C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 159 disconnected components.
 There are 100 islands with ids: 0, 3, 4, 5, 6, 8, 9, 25, 30, 34, 35, 36, 40, 47, 48, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 77, 78, 79, 80, 81, 86, 117, 124, 128, 129, 130, 154, 168, 169, 170, 173, 174, 175, 176, 177, 182, 183, 184, 185, 186, 187, 188, 191, 193, 195, 215, 231, 238, 239, 244, 248, 270, 277, 323, 328, 351, 357, 362, 369, 385, 388, 389, 390, 391, 392, 413, 418, 419, 420, 421, 429, 430, 432, 433, 434, 435, 436, 437, 447, 485, 506, 513, 573, 721, 722, 723, 726, 727, 728, 738.
  warnings.warn(message)
C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [6]:
import pandas as pd

In [7]:
tab_pop_u_centre_cand=pd.pivot_table(data_cl_centre,
                                     index='cluster_label',values='pop_built_s2',aggfunc='sum')

In [8]:
list_u_centre=tab_pop_u_centre_cand.query('pop_built_s2>50000').reset_index().cluster_label.tolist()

In [9]:
u_centre=data_cl_centre.query('cluster_label in @list_u_centre')

In [10]:
data_gpd['is_U_CENTRE']=data_gpd.Name.apply(lambda y: 1 if y in u_centre.Name.tolist() else 0)

In [11]:
#Filling gap of urbancentre => check 8 contiguity (queen)
init_u_centre=data_gpd.is_U_CENTRE.sum()
u_centre_up=0
print('N GRID WITH URBAN CENTRE:',init_u_centre)
while u_centre_up!=init_u_centre:
    init_u_centre=data_gpd.is_U_CENTRE.sum()
    wr = weights.contiguity.Queen.from_dataframe(data_gpd)
    data_gpd['neighbors']=wr.neighbors.values()
    temp=data_gpd.copy()
    for index, row in data_gpd.iterrows():
        if row['is_U_CENTRE']!=1:
            y_urbancentre=0
            if len(row['neighbors'])>4:
                for y in row['neighbors']:
                    if temp.loc[y,'is_U_CENTRE']==1:
                        y_urbancentre+=1
                if y_urbancentre>4:
                    data_gpd.at[index,'is_U_CENTRE']=1
    u_centre_up=data_gpd.is_U_CENTRE.sum()
    print('=====================')
    print('N GRID AFTER GAP FILLING:', u_centre_up)
print('==================')
u_centre=data_gpd.is_U_CENTRE.sum()
print('N GRID COMPLETED AFTER GAP FILLING',u_centre)

N GRID WITH URBAN CENTRE: 465


C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 538


C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 551


...

N GRID AFTER GAP FILLING: 557


C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 558


C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 2 disconnected components.
  warnings.warn(message)


N GRID AFTER GAP FILLING: 558
N GRID COMPLETED AFTER GAP FILLING 558


In [12]:
data_gpd=data_gpd[['Name','pop_built_s2','geometry','is_U_CENTRE']]

In [13]:
data_u_cluster=data_gpd.query('is_U_CENTRE!=1').query('pop_built_s2>=300')

In [14]:
wr = weights.contiguity.Queen.from_dataframe(data_u_cluster)
data_u_cluster['cluster_label']=wr.component_labels

C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 69 disconnected components.
 There are 20 islands with ids: 12, 88, 132, 141, 161, 163, 229, 287, 296, 416, 594, 723, 724, 755, 787, 886, 887, 888, 889, 890.
  warnings.warn(message)


In [15]:
tab_pop_u_cluster_cand=pd.pivot_table(data_u_cluster,
                                     index='cluster_label',values='pop_built_s2',aggfunc='sum')
list_u_cluster=tab_pop_u_cluster_cand.query('pop_built_s2>=5000').reset_index().cluster_label.tolist()

In [16]:
u_cluster=data_u_cluster.query('cluster_label in @list_u_cluster')

In [17]:
data_gpd['is_U_CLUSTER']=data_gpd.Name.apply(lambda y: 1 if y in u_cluster.Name.tolist() else 0)

In [18]:
data_gpd['classification']=data_gpd.apply(lambda y: 'CENTRE' 
                                          if y.is_U_CENTRE==1 else ('CLUSTER' if y.is_U_CLUSTER==1 else 'RURAL'),axis=1)

In [19]:
data_gpd.to_file('degurba_5100.gpkg',driver='GPKG')

C:\Users\User\Anaconda3\envs\pysal_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
